<a href="https://colab.research.google.com/github/SAMARTH-1301/Topsis-for-Pretrained-Models/blob/main/Topsis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

models_data = {
    'Model': ['BERT', 'RoBERTa', 'DistilBERT', 'ALBERT', 'XLNet'],
    'Accuracy': [0.92, 0.94, 0.89, 0.88, 0.91],
    'Training_Time': [4.5, 5.0, 2.5, 2.0, 4.0],
    'Model_Size': [1.2, 1.5, 0.5, 0.7, 1.1],
    'F1_Score': [0.91, 0.93, 0.88, 0.87, 0.90]
}

def apply_topsis(df, weights, criteria):

    decision_matrix = df.iloc[:, 1:].values

    normalized_matrix = decision_matrix / np.sqrt(np.sum(decision_matrix**2, axis=0))

    weighted_matrix = normalized_matrix * weights

    ideal = np.zeros(len(weights))
    negative_ideal = np.zeros(len(weights))

    for i in range(len(weights)):
        if criteria[i] == '+':
            ideal[i] = np.max(weighted_matrix[:, i])
            negative_ideal[i] = np.min(weighted_matrix[:, i])
        else:
            ideal[i] = np.min(weighted_matrix[:, i])
            negative_ideal[i] = np.max(weighted_matrix[:, i])

    s_positive = np.sqrt(np.sum((weighted_matrix - ideal)**2, axis=1))
    s_negative = np.sqrt(np.sum((weighted_matrix - negative_ideal)**2, axis=1))

    closeness = s_negative / (s_positive + s_negative)

    return closeness

def plot_results(df, scores):
    """Create a bar plot of TOPSIS scores"""
    plt.figure(figsize=(10, 6))
    plt.bar(df['Model'], scores)
    plt.title('TOPSIS Scores for Text Classification Models')
    plt.xlabel('Models')
    plt.ylabel('TOPSIS Score')
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.savefig('topsis_results.png')
    plt.close()

def main():

    df = pd.DataFrame(models_data)

    weights = np.array([0.3, 0.2, 0.2, 0.3])

    criteria = ['+', '-', '-', '+']

    topsis_scores = apply_topsis(df, weights, criteria)

    df['TOPSIS_Score'] = topsis_scores

    results = df.sort_values('TOPSIS_Score', ascending=False)

    plot_results(df, topsis_scores)

    print("\nTOPSIS Analysis Results:")
    print("-" * 50)
    print(results)
    print("\nBest Model:", results.iloc[0]['Model'])
    print("TOPSIS Score:", round(results.iloc[0]['TOPSIS_Score'], 4))

    return results

if __name__ == "__main__":
    results = main()


TOPSIS Analysis Results:
--------------------------------------------------
        Model  Accuracy  Training_Time  Model_Size  F1_Score  TOPSIS_Score
2  DistilBERT      0.89            2.5         0.5      0.88      0.866792
3      ALBERT      0.88            2.0         0.7      0.87      0.823046
4       XLNet      0.91            4.0         1.1      0.90      0.374654
0        BERT      0.92            4.5         1.2      0.91      0.258423
1     RoBERTa      0.94            5.0         1.5      0.93      0.102700

Best Model: DistilBERT
TOPSIS Score: 0.8668
